# SVM On 3 Data Sets

### In this notebook I will need to:
- load the .mat files with scipy.io
- normalize data
- visualize SVM with linear kernel and then Gaussian Kernel
- find the best C and sigma with GridSearchCV


In [11]:
# Setup environment
import scipy.io
import matplotlib.pyplot as plt 
import matplotlib
import pandas as pd
import numpy as np

In [43]:
# read in data sets 
from scipy.io import loadmat
data1 = loadmat('ex6data1.mat')
data2 = loadmat('ex6data2.mat')
data3 = loadmat('ex6data3.mat')

X1, y1 = data1['X'], data1['y']
X2, y2 = data2['X'], data2['y']
X3, y3 = data3['X'], data3['y']
X = [X1, X2, X3]
y = [y1, y2, y3]

In [44]:
# normalize features
from sklearn.preprocessing import normalize 

for i in np.arange(0, 3):
    X[i] = normalize(X[i])
    y[i] = normalize(y[i])
    
X1, X2, X3 = X
y1, y2, y3 = y

In [45]:
# Visualize 3 data sets